# Multiple answer cleanup
Attempt at dealing with fields that contain comma-separated lists of answers from questions with multiple-select answer options that also allow respondents to enter their own 'other' answer.

This notebook assumes the file named `responses_scrubbed_tagged.csv` has already been created by running the notebook named `free_text_tagging.ipynb`.  This notebook then creates a new data file called `responses_complete.csv` which has the complete set of final data.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, Latex

## Load tagged data set
This notebook assumes you have already scrubbed and tagged the data using the `free_text_tagging.ipynb` notebook first.

In [2]:
df = pd.read_csv('responses_scrubbed_tagged.csv', index_col=None, parse_dates=['date'])
df.sample(n=10) #random sample of rows

,date,neighborhood,demographic,child_bus_freq,child_bike_freq,child_driven_freq,child_drive_freq,child_walk_freq,walk_freq,bike_freq,...,no_bike_reason_2,no_bike_reason_3,drive_reason_0,drive_reason_1,drive_reason_2,drive_reason_3,drive_reason_4,drive_reason_5,problem_tags,suggestion_tags
293,2019-07-04 14:04:42,Upper Village,Senior,NaN,NaN,NaN,NaN,NaN,"Every day, or almost every day",Once every few days,...,NaN,NaN,My own personal preference,NaN,NaN,NaN,NaN,NaN,"shoprite,129,hughes, n riverside,speeding,miss...","reduce speed,enforce speed"
85,2019-07-03 16:14:37,Upper Village,Adult with school kid,Rarely or never,NaN,Once in a while,Rarely or never - for other reasons,Rarely or never - for other reasons,"Every day, or almost every day",Once in a while,...,Places to lock and leave my bike,NaN,I don't have time to walk or bicycle,Safety concerns with bicycling,Good places to leave and lock bike,NaN,NaN,NaN,"129,aggressive driving,driver awareness,speedi...",reduce speed
63,2019-07-03 15:56:22,NaN,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once in a while,Once in a while,...,NaN,NaN,My own personal preference,NaN,NaN,NaN,NaN,NaN,"missing sidewalks, road surface,poor lighting","more sidewalks, better lights"
196,2019-07-03 20:05:42,Half Moon Bay,Adult with school kid,"Every day, or almost every day",Rarely or never - for other reasons,Rarely or never,Rarely or never - my children are too young,Rarely or never - my children are too young,Once every few days,Rarely or never,...,NaN,NaN,Time,distance,weather. We do walk places sometimes,NaN,NaN,NaN,"driver awareness,municipal place,missing sidew...","more sidewalks,enforce crosswalks"
314,2019-07-04 21:16:05,Harmon,Senior,NaN,NaN,NaN,NaN,NaN,Once every few days,Rarely or never,...,NaN,NaN,I don't have time to walk or bicycle,NaN,NaN,NaN,NaN,NaN,"missing sidewalks,sidewalk condition","more sidewalks,maintain sidewalks"
405,2019-07-08 19:25:43,Old Post Road North,Senior,NaN,NaN,NaN,NaN,NaN,Once every few days,Once every few days,...,NaN,NaN,My own personal preference,NaN,NaN,NaN,NaN,NaN,"speeding, s riverside, n riverside","enforce speed, bike lanes"
423,2019-07-09 13:27:16,Half Moon Bay,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once every few days,Once in a while,...,NaN,NaN,Safety concerns with bicycling,NaN,NaN,NaN,NaN,NaN,NaN,bike lanes
33,2019-07-03 15:44:54,Upper Village,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once every few days,Rarely or never,...,NaN,NaN,I’m frequently transporting a small child.,NaN,NaN,NaN,NaN,NaN,"sidewalk condition,cpa,five corners,aggressive...","bike lanes,more traffic lights,improve route 9..."
260,2019-07-04 09:21:15,Harmon,Adult without school kid,NaN,NaN,NaN,NaN,NaN,"Every day, or almost every day",Once in a while,...,Bad behaviour by males. Police won't do anything.,NaN,Safety concerns with walking,Safety concerns with bicycling,NaN,NaN,NaN,NaN,driver awareness,bike lanes
206,2019-07-03 20:52:31,Upper Village,NaN,NaN,NaN,NaN,NaN,NaN,Once every few days,Rarely or never,...,NaN,NaN,My own personal preference,NaN,NaN,NaN,NaN,NaN,"cpa,dpw,num cars","reduce num cars, carpool"


## Answer options to each question
A dictionary containing lists of all the questions and the answer options that the user was asked to select from for each question in the Google Form.

In [3]:
# a dictionary of columns that contain comma-separated multiple answer options
multiple_answer_options = {
    'schools': ['CET', 'PVC', 'CHHS', 'Homeschooled'],
    'child_drive_reason': ['No - we do not drive, or prefer not to drive', 'Our own personal preference', 'Lack of available busing where we live', 'The bus schedule does not match our schedule', 'Safety concerns with buses', 'Safety concerns with walking', 'Safety concerns with bicycling', "My child's health condition"],
    'child_no_walk_reason': ['No - they walk a lot', 'My child does not like to walk', 'We live too far to walk', "We don't have time to walk", 'Fear of dangerous driving', 'Lack of adequate sidewalks', 'Lack of adequate crosswalks at busy intersections', 'Lack of crossing guards at busy intersections', "My child's health condition", 'Visually unappealing route'],
    'child_no_bike_reason': ['No - they bicycle a lot', 'My child does not like to bicycle', 'My child is too young to bicycle', 'We live too far to bicycle', 'Our own personal preference', 'Fear of dangerous driving', 'Lack of adequate bike lanes', "My child's health condition", 'Visually unappealing route', 'Hills'],
    'no_walk_reason': ['No - I walk a lot', 'I live too far to walk', 'I do not like to walk', "I don't have time to walk", 'Fear of dangerous driving', 'Lack of adequate sidewalks', 'Lack of adequate crosswalks at intersections', 'Lack of crossing guards at intersections', "My health condition", 'My own personal preference', 'Visually unappealing'], 
    'no_bike_reason': ['No - I bicycle a lot', 'I live too far to bicycle', 'I do not like to bicycle', "I don't have time to bicycle", 'Fear of dangerous driving', 'Lack of adequate bike lanes', 'My health condition', 'My own personal preference', 'Visually unappealing'],
    'drive_reason': ['No - I do not drive, or prefer not to drive', "I don't have time to walk or bicycle", 'Safety concerns with walking', 'Safety concerns with bicycling', 'My own health condition', 'My own personal preference']
}

# remember that there was also an 'other' field where users could enter custom answers... we'll deal with that later.

## Fix the mistake we made splitting these columns
In the `cleanup.ipynb` notebook, we split up columns with multiple answer options into multiple columns, such as `child_drive_reason_0`, `child_drive_reason1`, etc.  Merge them back together here.

In [5]:
# loop through each question title
for q in multiple_answer_options.keys():
    column_list = []
    for i in range(10):
        column_name = q + '_' + str(i) #auto-generate numbered column name
        if column_name in df.columns:
            column_list.append(column_name)
    if len(column_list) > 0:
        # create a new column with all answers comma-separated
        df[q] = df[column_list].apply(lambda x: ', '.join(x.fillna('').map(str)), axis=1)
        df[q] = df[q].str.strip(', ')
        
        # delete the old numbered columns
        df = df.drop(column_list, axis=1)        
df.columns

Index(['date', 'neighborhood', 'demographic', 'child_bus_freq',
       'child_bike_freq', 'child_driven_freq', 'child_drive_freq',
       'child_walk_freq', 'walk_freq', 'bike_freq', 'bikes_on_sidewalk',
       'self_jog_frequency', 'commutes', 'child_self_school',
       'child_self_bus_freq', 'child_self_bike_freq', 'child_self_driven_freq',
       'child_self_drive_freq', 'child_self_walk_freq', 'child_self_commutes',
       'child_self_has_children', 'commuter_distance', 'commuter_type',
       'commuter_walk_to_station_freq', 'commuter_bike_to_station_freq',
       'commuter_drive_to_station_freq', 'commuter_carpool_to_station_freq',
       'commuter_driven_to_station_freq', 'commuter_bus_to_station_freq',
       'feelings', 'problem_areas', 'drivers_are_safe', 'bicyclists_are_safe',
       'suggested_improvements', 'additional_comments', 'owns_business',
       'business_type', 'business_space', 'business_pedestrian_synergy',
       'business_bicyclists_synergy', 'business_bike_r

## Add in the tagging data
Tagging is done by the data analyst, and is assumed to be correct.  From here on, for simplicity, it will from here be stored and analyzed the same as if tags had been a question with multiple answer options with the most popular tags being treated as the answer options.

In [6]:
#add the most popular complaint tags to the dictionary of questions and their answer options
multiple_answer_options['problem_tags'] = [ 'speeding', 'missing sidewalks', 'cpa', 'sidewalk condition', 'aggressive driving', 'driver awareness', 's riverside', '129', 'maple', 'cyclists breaking rules', 'cleveland', 'road surface', 'municipal place', 'missing crosswalks', 'bushes on sidewalk', 'grand', 'narrow roads', 'mt airy', 'shoprite', 'benedict', 'five corners', 'van wyck', 'dummy light', 'crossing guard', 'gottwald circle', 'n riverside', 'poor lighting', 'croton commons', 'truesdale', 'hmb trail', 'cet', 'on-street parking', 'old post n', 'olcott', 'old post s', 'morningside' ]

#add the most popular suggestion tags to the dictionary of questions and their answer options
multiple_answer_options['suggestion_tags'] = [ 'bike lanes', 'more sidewalks', 'enforce speed', 'maintain sidewalks', 'reduce speed', 'educate cyclists', 'more crosswalks', 'speed bumps', 'no bike lanes', 'maintain crosswalks', 'enforce crosswalks', 'sidewalk on mt airy s', 'reduce on-street parking', 'more traffic lights', 'better lighting', 'more stop signs', 'dedicated pedestrian walk signals', 'more signs', 'improve route 9 access', 'educate pedestrians', 'outlaw cycling', 'enforce trimming of bushes', 'speed cameras', 'parking on one side only', 'bike racks', 'traffic calming', 'speed indicator signs' ]


## Limit to just the multiple select columns
For debugging

In [7]:
column_list = list(multiple_answer_options.keys())
df = df[column_list]
df = df.fillna('')
df.sample(n=10) #random sample of rows

,schools,child_drive_reason,child_no_walk_reason,child_no_bike_reason,no_walk_reason,no_bike_reason,drive_reason,problem_tags,suggestion_tags
116,"CET, PVC",Kids running late,We don't have time to walk,"Fear of dangerous driving, Lack of adequate bi...","My health condition, My own personal preferenc...",I don't have time to bicycle,"My own personal preference, Hills","municipal place, cpa","improve route 9 access,bike lanes"
268,,,,,My own personal preference,My health condition,"Safety concerns with bicycling, My own persona...","crossing guard,missing curbs,sidewalk conditio...","educate cyclists,retrain crossing guards"
469,,,,,No - I walk a lot,No - I bicycle a lot,Safety concerns with bicycling,"s riverside,benedict,sidewalk condition","maintain sidewalks,bike lanes"
130,"PVC, CHHS",Our own personal preference,My child does not like to walk,Our own personal preference,"Timing, need more time",Timing I am in a rush and I have a nice bike t...,Timing and usually in a rush,taxes,lower taxes
285,,,,,No - I walk a lot,Lack of adequate bike lanes,"No - I do not drive, or prefer not to drive, C...","n riverside,missing crosswalks,bank st,missing...","maintain sidewalks,more crosswalks,more street..."
76,CET,"No - we do not drive, or prefer not to drive",Fear of dangerous driving,Fear of dangerous driving,My own personal preference,I do not like to bicycle,My own personal preference,"speeding,maple","enforce speed, reduce speed"
155,,,,,I don't have time to walk,Fear of dangerous driving,I don't have time to walk or bicycle,,maintain sidewalks
537,"PVC, CHHS",Only in times of bad weather as Mt. Airy is no...,No - they walk a lot,Our own personal preference,No - I walk a lot,I do not like to bicycle,Safety concerns with walking,"missing sidewalks, mt airy","more signs, sidewalk on mt airy s, more sidewalks"
174,,,,,My own personal preference,"Hazards on the road: debris, uneven surfaces etc.",My own personal preference,"gottwald circle,cleveland,road surface,s river...","parking on one side only,reduce on-street park..."
339,,,,,,,,"bushes on sidewalk,sidewalk condition,grand,mu...","more sidewalks,maintain sidewalks"


## Remove commas from any given answer option so we can later split by comma cleanly

In [8]:
# loop through all questions that allow more than one answer
for question, answer_list in multiple_answer_options.items():
    # loop through every answer in the answer list for this question
    for answer in answer_list:
        # remove the commas, from each answer, if any
        if ',' in answer:
            # update the answers with a cleaned version
            df[question] = df[question].str.replace(answer, answer.replace(',', '') )
            
    # remove commas from the list of answers in the question/answer dictionary
    multiple_answer_options[question] = [answer.replace(',' , '') for answer in answer_list]

# show the cleaned up answer options
#multiple_answer_options

## Tally answers
Indicate which users selected which answer options, and put any freeform text answers that users entered by clicking the 'other' option in Google Forms into '_other' columns.

In [9]:
def categorize_answers(response, question, answer_list):    
    """
    Takes a given set of responses to a multi-select question, and puts them into a well-formated Series of dummy values.
    
    @param response A list of the user's responses to this question
    @param question The title of the question (these are codes)
    @param answer_list A list of the answer options that were presented to the user in the Google Form
    """
    
    # set up some lists that we will use to collect responses as dummy values
    indices = [] # will contain the text of the answer option that the user selected
    values = [] # will contain dummy values (1s) for each answer option the user selected, NaNs otherwise.
    
    # skip any nan or blank values
    if str(response) != 'nan' and type(answer_list) != float and type(question) != float and type(response) != float:
        #display(question + " - " + str(response))
        for answer_option in response:
            answer_option = answer_option.strip() # remove any leading/trailing whitespace
            if answer_option in answer_list and question + '_' + answer_option not in indices:
                indices.append(question + '_' + answer_option)
                values.append(1)
            elif answer_option.strip() != '' and question + '_' + 'other' not in indices:
                indices.append(question + '_' + 'other')
                values.append(answer_option)
    
        # put it all into a pandas Series
        response = pd.Series(values,index=indices).fillna(0).astype(str)
        
    return response

# slice up answers by comma and give each its own column
df2 = pd.DataFrame() # build a new dataframe from scratch

# loop through each question/answer item
for question, answer_list in multiple_answer_options.items():
    # get a new dataframe with the questions as indexes and the valuess as dummy values indicating user selections
    d = df[question].str.split(',\s*').apply(categorize_answers, question=question, answer_list=answer_list)
    df2 = df2.append(d) # append this as a row to the new dataframe

# show the new dataframe
df2.sample(n=10) #random sample of rows

,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_drive_reason_other,child_no_bike_reason_Fear of dangerous driving,...,suggestion_tags_other,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Convert values to ints or booleans, where possible
Most columns contain NaNs or 1s.  NaNs are converted to 0s then to booleans, if desired.  Integer dummy values are perhaps more useful so we can take counts of responses by adding the ints up.  The 'other' columns contain text and are left alone.

In [10]:
# loop through each column
for col in df2.columns:
    try:
        # try to convert to int
        df2[col] = df2[col].fillna(0).astype(int)#.astype(bool)
    except:
        # if this column contains text, skip...
        continue

## Group rows by 'id' so there is one row for each respondent

In [11]:
# add up all values in each column to get a full set of values or each id
# this loses the 'other' columns, since they are not numeric
df3 = df2.groupby(df2.index).sum()

# show the new grouped data... note that it is missing the 'other' fields... more on that later
df3.sample(n=10) #random sample of rows

,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_no_bike_reason_Fear of dangerous driving,child_no_bike_reason_Hills,...,suggestion_tags_no bike lanes,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
25,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
432,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
304,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
411,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
174,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0
360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
94,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
364,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Combine the selected answer options with the 'other' responses users gave

In [12]:
# loop through all questions
for question, answer_options in multiple_answer_options.items():
    # get a dataframe of just the 'other' answers to this question
    other_df = df2[pd.notnull(df2[question + '_other'])][question + '_other'].to_frame()
    # concatenate those responses to the original dataframe
    df3 = df3.join(other_df)
    
# show the data
df3.sample(n=10) #random sample of rows

,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_no_bike_reason_Fear of dangerous driving,child_no_bike_reason_Hills,...,suggestion_tags_traffic calming,schools_other,child_drive_reason_other,child_no_walk_reason_other,child_no_bike_reason_other,no_walk_reason_other,no_bike_reason_other,drive_reason_other,problem_tags_other,suggestion_tags_other
337,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wells,make wells one way
547,0,0,0,0,0,0,0,1,1,0,...,0,NaN,CET has 7:35-7:40 Chorus and Band practice. T...,Well,She still rides on the side walk,NaN,I don't own a bike,NaN,bushes on sidewalks,NaN
52,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gerstein,extend croton landing path to peekskill
437,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,Picking up groceries or at night,NaN,NaN
446,0,0,0,0,0,0,0,0,0,1,...,0,NaN,Bad weather,NaN,NaN,NaN,NaN,NaN,NaN,sidewalk condition
238,0,0,0,1,0,0,0,0,0,0,...,0,NaN,NaN,Timing or Bc of carrying too many school supplies,NaN,NaN,NaN,NaN,NaN,NaN
257,0,0,0,0,0,0,0,0,0,0,...,0,Currently pre-school but soon CET,Her pre-school is out of town,Again,She's just not biking yet.,NaN,Can't bike with kiddo,NaN,gerstein,NaN
392,0,0,0,1,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cyclists at croton landing,NaN
132,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,Sometimes I walk,NaN,NaN


## Order the columns alphabetically
So it looks a bit more orderly

In [13]:
column_names = df3.columns.sort_values()
df3 = df3[column_names]
df3.sample(n=10) #random sample of rows

,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_drive_reason_other,child_no_bike_reason_Fear of dangerous driving,...,suggestion_tags_other,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
271,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
219,0,0,1,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
267,0,0,0,0,0,0,0,0,NaN,0,...,5 way stop on mt airy s,0,0,0,0,0,0,0,0,0
70,0,0,1,0,0,0,0,0,NaN,1,...,NaN,0,0,0,0,0,1,0,0,0
322,0,0,0,0,0,0,0,0,G Days and High school schedule.,1,...,open gate on highland during rush,0,0,0,0,1,0,0,0,0
509,0,0,1,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
153,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,1,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
15,0,0,0,0,1,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0


## Merge with remaining data

In [14]:
# load up the original data from scratch again
df = pd.read_csv('responses_scrubbed_tagged.csv', index_col=None, parse_dates=['date'])

# merge the new columns with the old dataframe based on the indexes of each row
df_complete = pd.merge(df, df3, right_index=True, left_index=True)

df_complete.sample(n=10) #random sample of rows

,date,neighborhood,demographic,child_bus_freq,child_bike_freq,child_driven_freq,child_drive_freq,child_walk_freq,walk_freq,bike_freq,...,suggestion_tags_other,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
30,2019-07-03 15:42:44,Old Post Road North,Adult without school kid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
255,2019-07-04 08:34:38,Harmon,Adult with school kid,"Every day, or almost every day",Rarely or never - for other reasons,Once every few days,Rarely or never - my children are too young,Once in a while,Once every few days,Once in a while,...,NaN,0,0,0,0,0,0,0,0,0
206,2019-07-03 20:52:31,Upper Village,NaN,NaN,NaN,NaN,NaN,NaN,Once every few days,Rarely or never,...,reduce num cars,0,0,0,0,0,0,0,0,0
54,2019-07-03 15:53:54,Upper Village,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once every few days,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
41,2019-07-03 15:47:08,Harmon,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once every few days,Rarely or never,...,stop signs on lexington near sunset park,0,0,0,0,0,0,0,0,0
375,2019-07-07 20:01:09,Upper Village,Adult with school kid,"Every day, or almost every day",Rarely or never - my children are too young,Rarely or never,Rarely or never - my children are too young,Rarely or never - my children are too young,Once every few days,Rarely or never,...,bike share,0,0,0,0,0,0,0,0,0
453,2019-07-10 13:59:38,Upper Village,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Rarely or never,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
248,2019-07-04 07:21:05,Old Post Road North,Senior,NaN,NaN,NaN,NaN,NaN,Once every few days,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
61,2019-07-03 15:56:08,Old Post Road North,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Rarely or never,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
494,2019-07-14 18:51:41,Harmon,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once every few days,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0


## Save to new CSV file

In [15]:
df_complete.to_csv('responses_complete.csv', index=False)

## Done!